In [0]:
!pip install -q pydrive
!pip install -q featexp

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd
import numpy as np
import featexp as ftp

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nnutil_module = drive.CreateFile({'id':'1oRft64GKmE9T3ibCaN5YsUL8ifNQsY3m'})
nnutil_module.GetContentFile('regr_prepare_data.py')

import regr_prepare_data as rpd

In [0]:
# Высчитывает для каждого значения признака средние величины target. Заменяет значения признаков
# на посчитанные средние. Меняет тип пизнака на float
def features_totarget_units(df_with_target, target, include = 'None', apply = False):
  dict = {}

  dropped_attribs, df_with_target = rpd.varianceThreshold(df_with_target, 0)
  
  for f in df_with_target:
    if f == target or (include != 'None' and f not in include):
      continue

    features = df_with_target.filter(items = [f, target])
    gb = features.groupby([f])
    gb = gb.mean()

    for i in range(gb.index.shape[0]):
      if apply and df_with_target[f].dtype != 'object':
        df_with_target[f] = df_with_target[f].apply(lambda x: gb.values[i] if x == gb.index[i] else x)
        df_with_target[f] = df_with_target[f].astype('float')
      dict[(gb.index.name, gb.index[i])] = gb.values[i][0]

  return dict, df_with_target

In [0]:
data = pd.read_csv('https://raw.githubusercontent.com/motokazmin/kaggle/ML_Boot_Camp/Performance_evaluation/data/x_train.csv')
lables = pd.read_csv('https://raw.githubusercontent.com/motokazmin/kaggle/ML_Boot_Camp/Performance_evaluation/data/y_train.csv')


full_data = pd.concat((data, lables), axis=1)
get_univariate_plots(full_data, 'time')

dict, full_data = features_totarget_units(full_data, target = 'time', apply = False)


In [0]:
print(dict)
print(full_data.info())